In [ ]:
pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d andrewmvd/heart-failure-clinical-data

In [ ]:
!unzip /content/heart-failure-clinical-data.zip -d /content/heartfailure/

Archive:  /content/heart-failure-clinical-data.zip
replace /content/heartfailure/heart_failure_clinical_records_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("/content/heartfailure/heart_failure_clinical_records_dataset.csv")

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
fig, ax = plt.subplots(figsize=(4, 5))
plt.scatter(df['age'],df['DEATH_EVENT'])

In [ ]:
sns.barplot(x=df['age'],y=df['DEATH_EVENT'])

In [ ]:
df.columns

In [ ]:
sns.barplot(x=df['age'],y=df['sex'],hue='DEATH_EVENT',data=df)

In [ ]:
df.info()

In [ ]:
df.isnull().any()

In [ ]:
df.describe()

In [ ]:
fig = plt.subplots(figsize=(15,15))
sns.heatmap(df.corr(),annot=True,fmt='.1f')

In [ ]:
df.drop(['time','ejection_fraction','serum_sodium'],axis=True,inplace=True)

In [ ]:
df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = df.drop(['DEATH_EVENT'],axis=1)
y = df.DEATH_EVENT

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [ ]:
# Load Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb



In [ ]:
#  model evaluation function
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
from pandas.plotting import scatter_matrix
from matplotlib import pyplot

In [ ]:
from xgboost.sklearn import XGBClassifier

models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('KNN',KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('ADA', AdaBoostClassifier()))
models.append(('XGB', XGBClassifier()))

In [ ]:
results =[]
names = []

for name, model in models:
    kfold = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)
    cv_result = cross_val_score(model, x_train,y_train, cv=kfold , scoring='accuracy')
    results.append(cv_result)
    names.append(name)
    print("cv_result", cv_result)
    print('%s: %f (%f)'% (name, cv_result.mean(), cv_result.std()))
    print("")

In [ ]:
# compare Algorithms
pyplot.boxplot(results, labels=names)
pyplot.title('Algorithm Comparison')
pyplot.show()

In [ ]:
# Make predictions on test dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
model = LogisticRegression()
model.fit(x_train_scaled, y_train)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
# Evaluate predictions
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

In [ ]:
# Make predictions on test dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
model = RandomForestClassifier(n_estimators=200,random_state=42)
model.fit(x_train_scaled, y_train)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
# Evaluate predictions
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()
param_grid = [
    {"n_estimators": [10, 100, 200, 500], "max_depth": [None, 5, 10], "min_samples_split": [2, 3, 4]}
]

grid_search = GridSearchCV(clf, param_grid, cv=3, scoring="accuracy", return_train_score=True)

grid_search.fit(x_train_scaled, y_train)


In [ ]:
prod_final_clf = grid_search.best_estimator_

In [ ]:
prod_final_clf

In [ ]:
predictions = prod_final_clf.predict(x_test)

In [ ]:
predictions

In [ ]:
n_estimators_values = [10, 50, 100, 150, 200]
accuracy_scores = []
for n_estimators in n_estimators_values:
    # Create the Random Forest Classifier
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

    # Train the model on the training data
    rf_classifier.fit(x_train, y_train)

    # Make predictions on the test data
    y_pred = rf_classifier.predict(x_test)

    # Calculate the accuracy of the model and append to the list
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Plot the accuracy scores
plt.plot(n_estimators_values, accuracy_scores, marker='o')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Number of Trees in Random Forest')
plt.grid(True)
plt.show()